## Data Cleaning

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xlrd

based on the bug reported here: https://www.kaggle.com/c/sberbank-russian-housing-market/discussion/34364

In [8]:
train_df = pd.read_csv("input/train.csv")
test_df = pd.read_csv("input/test.csv")
fix_df = pd.read_excel("input/BAD_ADDRESS_FIX.xlsx").drop_duplicates('id').set_index('id')
macro_df = pd.read_csv('input/macro.csv')

train_df.update(fix_df)
test_df.update(fix_df)

#train_df = train_df.drop(['product_type'], axis=1)
#test_df = test_df.drop(['product_type'], axis=1)

print ("Train data shape: ", train_df.shape)
print("Test data shape: ",test_df.shape)

print('Fix in train: ', train_df.index.intersection(fix_df.index).shape[0])
print('Fix in test : ', test_df.index.intersection(fix_df.index).shape[0])

Train data shape:  (30471, 292)
Test data shape:  (7662, 291)
Fix in train:  550
Fix in test :  115


# Important notes regarding features

THESE ARE SOME OF THE SUMMARY NOTES GAINED FROM KAGGLE DISCUSSIONS, QUESTIONS AND ANSWERS FROM SBERBANK

- CHECK LIFE SQ, FULL SQ, KITCH SQ FOR CONSISTENCY
- BUILD YEAR CAN BE IN FUTURE - PRE INVESTMENT TYPE 
- BUILD YEAR 0 AND 1 ARE MISTAKES 
- CHECK TRAIN AND TEST PRODUCT TYPES 
- CHECK NUM OF ROOMS FOR CONSISTENCY 
- MATERIAL EXPLAINED: 1 - panel, 2 - brick, 3 - wood, 4 - mass concrete, 5 - breezeblock, 6 - mass concrete plus brick
- STATE EXPLAINED: 4 BEST 1 WORST
- KITCHEN INCLUDED IN LIFE SQ CHECK INCONSISTENCY 
- FULL SQ > LIFE SQ (MOST PROBABLY) 
- KM DISTANCES ARE AIRLINE DISTANCES
- RAION POPUL AND FULL ALL ARE SAME CALC FROM DIFF SOURCES


## CORRECTIONS RULES FOR Full_SQ AND Life_SQ (APPLY TO TRAIN AND TEST):

- IF Life_Sq >= Full_Sq MAKE Life_Sq np.NaN
- IF Life_Sq < 5 np.NaN
- IF Full_Sq < 5 np.NaN 
- Kitch_Sq < Life_Sq
- IF Kitch_Sq == 0 OR 1 np.NaN
- CHECK FOR OUTLIERS IN Life_Sq, Full_Sq AND Kitch_Sq (too large or too small)
- Life_Sq / Full_Sq MUST BE CONSISTENT, it should be realistic a house without living area is impractical (0.3 IS A CONSERVATIVE RATIO)

In [9]:
test_df['kitch_sq'].head()

0     8.9
1     1.0
2     4.8
3    62.8
4     1.0
Name: kitch_sq, dtype: float64

In [10]:
'''
Data points where living area size is greater than total area then it is illogical,
hence its better to make living area as NaN.
This is applied to both test and train data.
''' 

bad_index = train_df[train_df.life_sq > train_df.full_sq].index
train_df.ix[bad_index, "life_sq"] = np.NaN

equal_index = [601,1896,2791]
test_df.ix[equal_index, "life_sq"] = test_df.ix[equal_index, "full_sq"]

bad_index = test_df[test_df.life_sq > test_df.full_sq].index
test_df.ix[bad_index, "life_sq"] = np.NaN

'''
Also it is impratical to live inside a house that is very small as small as 5 sq-m.
Therefore, change such points in train and test set to NaN
'''
bad_index = train_df[train_df.life_sq < 5].index
train_df.ix[bad_index, "life_sq"] = np.NaN

bad_index = test_df[test_df.life_sq < 5].index
test_df.ix[bad_index, "life_sq"] = np.NaN

bad_index = train_df[train_df.full_sq < 5].index
train_df.ix[bad_index, "full_sq"] = np.NaN

bad_index = test_df[test_df.full_sq < 5].index
test_df.ix[bad_index, "full_sq"] = np.NaN

'''
Looks like at index=13117 the kitchen size is 1970 sq-m while build year is NA. This data doesn't make 
sense as the total area is 31 sq-m and kitchen is exceeding this by a very large margin. By a quick look
at the build year feature it makes sense to have 1970 instead of kitchen size.
'''
kitch_is_build_year = [13117]
train_df.ix[kitch_is_build_year, "build_year"] = train_df.ix[kitch_is_build_year, "kitch_sq"]

'''
Once again, fixing up impractical kitchen sizeas with NaN 
'''
bad_index = train_df[train_df.kitch_sq >= train_df.life_sq].index
train_df.ix[bad_index, "kitch_sq"] = np.NaN

bad_index = test_df[test_df.kitch_sq >= test_df.life_sq].index
test_df.ix[bad_index, "kitch_sq"] = np.NaN

bad_index = train_df[(train_df.kitch_sq == 0).values + (train_df.kitch_sq == 1).values].index
train_df.ix[bad_index, "kitch_sq"] = np.NaN

bad_index = test_df[(test_df.kitch_sq == 0).values + (test_df.kitch_sq == 1).values].index
test_df.ix[bad_index, "kitch_sq"] = np.NaN

'''
Removing outliers (very large values in total area and those )
'''
bad_index = train_df[(train_df.full_sq > 210) & (train_df.life_sq / train_df.full_sq < 0.3)].index
train_df.ix[bad_index, "full_sq"] = np.NaN

bad_index = test_df[(test_df.full_sq > 150) & (test_df.life_sq / test_df.full_sq < 0.3)].index
test_df.ix[bad_index, "full_sq"] = np.NaN

'''
Removing outliers (very large values in living area)
'''
bad_index = train_df[train_df.life_sq > 300].index
train_df.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN

bad_index = test_df[test_df.life_sq > 200].index
test_df.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN



C:\Continuum\Anaconda3\envs\tf_cpu\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  
C:\Continuum\Anaconda3\envs\tf_cpu\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()


## Correction rules for Number of Rooms

- Check for outlier and eliminate them.
- A very small or large number, check if it is practical.
- Life_Sq / room > min room Sq (let's say 5 Sq for a room might be ok)
- If num room == 0 set to np.NaN
- Detect abnormal num rooms given for a Life_Sq and Full_Sq

In [11]:
'''
If the number of rooms are zero, set them as NaN
'''
bad_index = train_df[train_df.num_room == 0].index 
train_df.ix[bad_index, "num_room"] = np.NaN

bad_index = test_df[test_df.num_room == 0].index 
test_df.ix[bad_index, "num_room"] = np.NaN
'''
A minimum room size where anyone can live could be 5 Sq-m anything below could be suffocating or bad design, 
hence eliminate the rooms which are smaller than min size.
'''
bad_index = train_df[train_df.life_sq/train_df.num_room <= 5].index
train_df.ix[bad_index, "num_room"] = np.NaN

bad_index = test_df[test_df.life_sq/test_df.num_room <= 5].index
test_df.ix[bad_index, "num_room"] = np.NaN

## Floor and Max Floor; Material and State

- CHECK FLOOR < MAX FLOOR (IF FLOOR > MAX FLOOR -> MAX FLOOR np.NaN)
- CHECK FOR OUTLIERS

In [12]:
'''
If floor number of the building is greater than maximum number of floors
'''
bad_index = train_df[train_df.floor > train_df.max_floor].index
train_df.ix[bad_index, "max_floor"] = np.NaN

bad_index = test_df[test_df.floor > test_df.max_floor].index
test_df.ix[bad_index, "max_floor"] = np.NaN


In [13]:
'''
apartment condiditon is mainly from 1 to 4, clearly 33 doesn't fit in. 
'''
bad_index = train_df[train_df.state == 33].index
train_df.ix[bad_index, "state"] = np.NaN

## Saving cleaned test and trained data

In [15]:
test_df.to_csv("input/test_clean.csv", index= False, encoding= "utf_8")
train_df.to_csv("input/train_clean.csv", index = False, encoding= "utf_8")